In [3]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import numpy as np
replacement_value = 'dummy_repalcement_value'
path = '/home/td/Documents'

# Problem structure

One way to set up the problem is to make it a binary classification task. This has the advantages of simplicity but it makes the assumption that large Makes share characteristics which is not always true. In this pass I solved the problem this way. 

A more thorough way to set up the problem would be to predict the make or a group of makes sharing similar characteristics and back into the probability. 

# Data problems

In [4]:
# Label problems
path = '/home/td/Documents'
df = pd.read_csv('{path}/tickets.csv'.format(path=path), low_memory=False)

top_25_makes = df['Make'].value_counts()[:25].index.tolist()
df['Make'].value_counts()[:25]

TOYT    721411
HOND    491961
FORD    382695
NISS    311324
CHEV    297076
BMW     199221
MERZ    177307
VOLK    149501
HYUN    133864
DODG    127764
LEXS    124508
KIA     101746
JEEP    100909
AUDI     84229
MAZD     79853
OTHR     72411
GMC      62391
CHRY     57317
INFI     56809
ACUR     52703
SUBA     46898
VOLV     42330
TOYO     40064
MITS     37842
CADI     34080
Name: Make, dtype: int64

Curation will be required, there is at least 1 duplicate with toyota and there are some invalid makes such as OTHR

In [5]:
df.loc[:,'Latitude'] = df.loc[:,'Latitude'].replace(99999.0, np.nan)
df.loc[:,'Longitude'] = df.loc[:,'Longitude'].replace(99999.0, np.nan)


In [6]:


percent_missing = df.isnull().sum()*100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
missing_value_df

,column_name,percent_missing
Ticket number,Ticket number,0.000000
Issue Date,Issue Date,0.006143
Issue time,Issue time,0.029601
Meter Id,Meter Id,73.991538
Marked Time,Marked Time,96.669739
RP State Plate,RP State Plate,0.008767
Plate Expiry Date,Plate Expiry Date,9.108706
VIN,VIN,99.813099
Make,Make,50.062606
Body Style,Body Style,0.101879


There are problems with the latitudes and longitudes, they don't make to standard coordinates. For example: Record with ticket id 1109139006 should have latitude and longitude of 34.156940, -118.435250. It has 6467477, 1880027 instead. Provided lat and long don't seem to map to standard formats.


In [7]:
#TODO: check for differences in test and train set

# Label analysis

In [8]:
df_features = pd.read_csv('feature_analysis.csv')

FileNotFoundError: [Errno 2] File b'feature_analysis.csv' does not exist: b'feature_analysis.csv'

In [ ]:
df_features.sort_values('model_feature_importance', ascending = False)


In [ ]:
df_labels = pd.read_csv('label_analysis.csv')
df_labels = df_labels[df_labels['Make'].isin(top_25_makes)]
df_counts= df_labels['Make'].value_counts()
from sklearn.preprocessing import StandardScaler

relevant_features = ['Fine amount', 
                     'plate_expiration_diff_ts', 
                     'lat_long_outlier_score', 
                     'route_dummy_replacement_value',
                     'color_GY',
                     'color_BL',
                     'color_BK',
                     'color_WT',
                     'color_BL',
                     'color_SL',
                     'route_00600',
                     'route_00500']


for i in relevant_features:
    scaler = StandardScaler()
    df_labels.loc[:, i] =  scaler.fit_transform(df_labels[i].values.reshape(-1, 1))
df_labels


df_group = df_labels[['Make'] + relevant_features].groupby('Make').mean()

In [ ]:
from sklearn.cluster import KMeans


cluster_alg = KMeans(n_clusters = 2)
df_group['cluster'] = cluster_alg.fit_predict(df_group[relevant_features])
df_group.sort_values('cluster')


In [ ]:
df_group.groupby('cluster').mean()
        

# Features

There are 3 general type of features: location, car characteristics and ticket/violation characteristics. 

The best location features are the Route and ticket location density (lat_long_outlier_score). 

The best car characteristics features are the Body type and the car color. 

The best violation features are the fine amount and time of the violation.

In [ ]:
df_copy = df.copy()
df_copy = df_copy.dropna(subset = ['Make'])
df_copy['target'] = df_copy['Make'].isin(top_25_makes).astype(int)
df_copy.loc[:, 'ticket_dt'] = pd.to_datetime(df_copy['Issue Date'], errors='coerce')
df_copy.loc[:, 'plate_expiration_diff_dt'] = pd.to_datetime(df_copy['Plate Expiry Date'].astype(int, errors = 'ignore'), format = '%Y%m', errors='coerce')
df_copy.loc[:, 'plate_expiration_diff_dt'] = df_copy['plate_expiration_diff_dt'] - df['plate_expiration_diff_dt']
df_copy.loc[:, 'plate_expiration_diff_ts'] = df_copy['plate_expiration_diff_dt'].values.astype(np.int64)


In [ ]:
top_states = df_copy['RP State Plate'].value_counts()[:8].index.tolist()
df_main_states = df_copy[df_copy['RP State Plate'].isin(top_states)]
df_main_states.groupby(['RP State Plate'])['target'].mean().sort_values()

In [ ]:
top_routes = df_copy['Route'].value_counts()[:8].index.tolist()
df_route = df_copy[df_copy['Route'].isin(top_routes)]
df_route.groupby(['Route'])['target'].mean().sort_values()

In [ ]:
top_body_styles = df_copy['Body Style'].value_counts()[:8].index.tolist()
df_body_styles = df_copy[df_copy['Body Style'].isin(top_body_styles)]
df_body_styles.groupby(['Body Style'])['target'].mean().sort_values()

In [ ]:
top_body_styles = df_copy['Issue time'].value_counts()[:8].index.tolist()
df_body_styles = df_copy[df_copy['Issue time'].isin(top_body_styles)]
df_body_styles.groupby(['Issue time'])['target'].mean().sort_values()

In [ ]:
df_labels.corr().sort_values('lat_long_outlier_score')

# Evaluation

In [ ]:
24072 269106 13273 129304